In [ ]:
using Plots, LaTeXStrings, LinearAlgebra, StaticArrays

In [ ]:
include("QuantumKRS.jl") 
include("HermanKlukKRS.jl")
include("SemiclassicalMeanValsKRS/SemiclassicalMeanValsKRS.jl");

# Herman-Kluk wavefunctions

In [ ]:
# quantum wavefunction at kick=k
α    = 0.6
kick = 60
N_qu = 2^10

M0   = 3.0
ħs   = [M0/80]

par = QuantumKRS.Params(α, kick, N_qu, M0, ħs)

ms_qu, θs_qu, ψs_qu_m, ψs_qu_θ = QuantumKRS.wave_at_k(par);

In [ ]:
# HK wavefunction at kick=k
N_sc   = N_qu
γ      = 1.0      # unbiased coherent state width
ntrajs = 100_000

par = HermanKlukKRS.Params(α, kick, N_sc, M0, ħs[1], γ, ntrajs)

@time m_sc, θ_sc, ψ_sc_m, ψ_sc_θ = HermanKlukKRS.wave_at_k(par);

In [ ]:
p1=plot(ms_qu[1], imag.(ψs_qu_m[1]), lw=1, m=true, ms=2, xlims=(M0-2,M0+2), xlabel=L"m",label="quantum")
plot!(m_sc, imag.(ψ_sc_m), lw=1, ls=:dash, m=:x, ms=3, label="HK")

p2=plot(ms_qu[1], real.(ψs_qu_m[1]), lw=1, m=true, ms=2, xlims=(M0-2,M0+2), xlabel=L"m",label="quantum")
plot!(m_sc, real.(ψ_sc_m), lw=1, ls=:dash, m=:x, ms=3, label="HK")

p3=plot(θs_qu[1], imag.(ψs_qu_θ[1]), lw=1, xlabel=L"\theta",label="quantum", bottom_margin=3Plots.mm)
plot!(θ_sc, imag.(ψ_sc_θ), lw=1, ls=:dash, label="HK")

p4=plot(θs_qu[1], real.(ψs_qu_θ[1]), lw=1, xlabel=L"\theta",label="quantum", bottom_margin=3Plots.mm)
plot!(θ_sc, real.(ψ_sc_θ), lw=1, ls=:dash, label="HK")

plot(p1, p2, p3, p4, size=(1200,600))

# Semiclassical Mean values

In [ ]:
# check that there are no filaments with complicated structure inside the detector
M0   = 3.0
Lₘₐₓ = 0.1e-1

X0   = SVector{2}(3.0, 2.0) # center of detector
σ    = 2^(-7/2)             # standard deviation
s    = 4                    # multiples of standard deviation to truncate result
R    = s*σ                  # effective radius of detector

α    = 4.0
kick = 3

@time θ0s, Lk  = SemiclassicalMeanValsKRS.get_Lk(-α, kick, M0, Lₘₐₓ, X0, R, false)

println("Number of filaments    : ", length(Lk))

θ_fins, _ = SemiclassicalMeanValsKRS.filament_or_finger(θ0s, Lk);
println("Number of fingers      : ", length(θ_fins)) 

In [ ]:
p1=plot( X0[2] .+ R*cos.(0:0.01:2π), X0[1] .+ R*sin.(0:0.01:2π), c=:red, lw=1, label=false)
[plot!(last.(L), first.(L), c=:black, lw=1, label=false) for L in Lk]
plot(p1, size=(500,500))

In [ ]:
# figure above looks good, so we move on to compute quantum mean vals as function of ħ
α    = 4.0
kick = 3
N    = 2^13

M0   = 3.0
ns   = 1:500
ħs   = M0 ./ ns

par = QuantumKRS.Params(α, kick, N, M0, ħs)

X0   = SVector{2}(3.0, 2.0) 
σ    = 2^(-7/2)            
s    = 4                    
R    = s*σ                  

mps = QuantumKRS.Detector(σ, X0, R)

Ds_qu  = QuantumKRS.mean_vals_at_k(par, mps);

In [ ]:
# and then SemiclassicalMeanValsKRS mean vals

Nθ       = 20
warnings = true
Lₘₐₓ     = 0.01e-1
warnings = true

dect = SemiclassicalMeanValsKRS.Detector(σ, X0, R)
pars = SemiclassicalMeanValsKRS.Params(-α, kick, M0, Lₘₐₓ, Nθ, ħs, warnings)

@time Ds_smv = SemiclassicalMeanValsKRS.get_means_smv(pars, dect);

In [ ]:
plot(ns, Ds_qu, lw=1.8, c=:black, alpha=0.5, label="quantum", size=(1200,300))
plot!(ns, Ds_smv, lw=0.7, label="SemiclassicalMeanValsKRS", c=:blue)

### Please cite the original paper in case you are using this code.